In [1]:
import numpy as np
execfile('midi_to_state.py')
daft_state = midiToNoteStateMatrix('daft_punk-one_more_time.mid')
len(daft_state)

2401

In [2]:
def encode_midi_states(state_mat):
    chord_to_idx = {}
    idx_to_chord = {}
    for sixteenth_note in state_mat:
        notes_only = [note[0] for note in sixteenth_note]
        note_idcs_set = tuple([i for i, x in enumerate(notes_only) if x == 1])
        if len(note_idcs_set) == 0: note_idcs_set = (None)
        if not note_idcs_set in chord_to_idx.keys():
            new_idx = len(chord_to_idx) + 1
            chord_to_idx[note_idcs_set] = new_idx
            idx_to_chord[new_idx] = note_idcs_set
        else:
            note_set_idx = chord_to_idx[note_idcs_set]
    return [chord_to_idx, idx_to_chord]

results = encode_midi_states(daft_state)
print (len(results[0]))
print (len(results[1]))
chord_to_idx = results[0]
idx_to_chord = results[1]

381
381


In [3]:
import random
randnum = random.sample(range(len(results[0])),1)[0]
print randnum
assert(results[0][results[1][randnum]] == randnum)

287


In [4]:
# now the set of chords is a library of possible classes
# return a num_notes x len_chord_set
def encode_song(midi_states, chord_to_idx):
    song_encoded = np.zeros((len(midi_states)+1, len(chord_to_idx)+1))
    for index in range(len(midi_states)):
        notes_only = [note[0] for note in midi_states[index]]
        note_idcs_set = tuple([i for i, x in enumerate(notes_only) if x == 1])
        if len(note_idcs_set) == 0: note_idcs_set = (None)
        library_idx = chord_to_idx[note_idcs_set]
        song_encoded.itemset((index,library_idx), 1)
    return song_encoded

encoded_song = encode_song(daft_state, results[0])
len(encoded_song)

2402

In [5]:
randnum = random.sample(range(len(encoded_song)),1)[0]
print randnum
assert(sum(encoded_song[randnum]) == 1)

146


In [6]:
chord_to_idx = results[0]
idx_to_chord = results[1]

maxlen = 40
step = 3
chunks = []
next_chords = []
for i in range(0, len(encoded_song) - maxlen, step):
    chunks.append(encoded_song[i: i + maxlen])
    next_chords.append(encoded_song[i + maxlen])

# len(chunks) should == len(daft_state)/step
assert(len(chunks) == len(next_chords))

X = np.array(chunks)
y = np.array(next_chords)
print(X.shape)
print(y.shape)
print(sum(y[0]))

(788, 40, 382)
(788, 382)
1.0


In [7]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM

# build the model: 2 stacked LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, len(chord_to_idx)+1)))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(chord_to_idx)+1))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

Using Theano backend.
Using gpu device 0: GRID K520 (CNMeM is disabled, cuDNN Version is too old. Update to v5, was 2000.)


Build model...


In [8]:
model.fit(X, y, batch_size=128, nb_epoch=100)

Epoch 1/100
788/788 [==============================] - 2s - loss: 5.6790 - acc: 0.0711     
Epoch 2/100
788/788 [==============================] - 2s - loss: 4.9951 - acc: 0.0749     
Epoch 3/100
788/788 [==============================] - 2s - loss: 4.7024 - acc: 0.0812     
Epoch 4/100
788/788 [==============================] - 2s - loss: 4.7192 - acc: 0.0622     
Epoch 5/100
788/788 [==============================] - 2s - loss: 4.7754 - acc: 0.0761     
Epoch 6/100
788/788 [==============================] - 2s - loss: 4.7486 - acc: 0.0520     
Epoch 7/100
788/788 [==============================] - 2s - loss: 4.6055 - acc: 0.0863     
Epoch 8/100
788/788 [==============================] - 2s - loss: 4.5032 - acc: 0.0838     
Epoch 9/100
788/788 [==============================] - 2s - loss: 4.4756 - acc: 0.0863     
Epoch 10/100
788/788 [==============================] - 2s - loss: 4.4689 - acc: 0.0863     
Epoch 11/100
788/788 [==============================] - 2s - loss: 4.4467 - acc

In [ ]:
def sample(a, temperature=1.0):
    # helper function to sample an index from a probability array
    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    return np.argmax(np.random.multinomial(1, a, 1))

start_index = random.randint(0, len(encoded_song) - maxlen - 1)

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = []
    song_chunk = encoded_song[start_index: start_index + maxlen]
    generated.append(song_chunk)
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()